# Utils
> Utility functions with wider use potential

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| exporti
import json, os
import itertools as it
from collections import defaultdict

import numpy as np
import pandas as pd
import datetime as dt

import altair as alt

from typing import List, Tuple, Dict, Union, Optional

In [ ]:
#| export
# Value or Default - returns key value in dict if key in dict, otherwise Mone
def vod(d,k,default=None): return d[k] if k in d else default

In [ ]:
#| export

# I'm surprised pandas does not have this function but I could not find it. 
def factorize_w_codes(s, codes):
    res = s.replace(dict(zip(codes,range(len(codes)))))
    if not s.isin(codes).all(): # Throw an exception if all values were not replaced
        vals = set(s) - set(codes)
        raise Exception(f'Codes for {s.name} do not match all values: {vals}')
    return res.to_numpy()

In [ ]:
#| export

# Simple batching of an iterable
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [ ]:
#| export

# turn index values into order indices
def loc2iloc(index, vals):
    d = dict(zip(np.array(index),range(len(index))))
    return [ d[v] for v in vals ]

In [ ]:
#| export

# Allow 'constants' entries in the dict to provide replacement mappings
# This leads to much more readable jsons as repetitions can be avoided
def replace_constants(d, constants = {}):
    if type(d)==dict and 'constants' in d:
        constants = constants.copy() # Otherwise it would propagate back up through recursion - see test6 below
        constants.update(d['constants'])
        del d['constants']

    for k, v in (d.items() if type(d)==dict else enumerate(d)):
        if type(v)==str and v in constants:
            d[k] = constants[v]
        elif type(v)==dict or type(v)==list:
            replace_constants(v,constants)

In [ ]:
# Test replace_constants
d = {
    'constants': { 'a': {'a':1}, 'b':['b'] },
    'test1': 'a',
    'test2': [1,'b'],
    'test3': { 'xy': 'a' },
    'test4': { 'xy': [2, 'b'] },
    'test5': { 'constants': {'a': ['a'] }, 'x':'a' },
    'test6': 'a'
}
replace_constants(d)
assert d == {'test1': {'a': 1}, 'test2': [1, ['b']], 'test3': {'xy': {'a': 1}}, 'test4': {'xy': [2, ['b']]}, 'test5': {'x': ['a']}, 'test6': {'a': 1}}

In [ ]:
#| export

# JSON encoder needed to convert pandas indices into lists for serialization
def index_encoder(z):
    if isinstance(z, pd.Index):
        return list(z)
    else:
        type_name = z.__class__.__name__
        raise TypeError(f"Object of type {type_name} is not serializable")

In [ ]:
#| export

# Helper function to turn a dictionary into an Altair scale (or None into alt.Undefined)
def to_alt_scale(scale):
    if scale is None: scale = alt.Undefined
    if isinstance(scale,dict):
        scale = alt.Scale(domain=list(scale.keys()),range=list(scale.values()))
    return scale

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()